In [1]:
import tensorflow as tf

model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)), tf.keras.layers.MaxPooling2D((2, 2)), tf.keras.layers.Flatten(), tf.keras.layers.Dense(10, activation='softmax')])

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data() 
x_train, x_test = x_train / 255.0, x_test / 255.0 

In [3]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

In [1]:
def train(model, x_train, y_train, rank, size):
    
    n = len(x_train)
    chunk_size = n // size 
    start = rank * chunk_size 
    end = (rank + 1) * chunk_size

    if rank == size - 1:
        end = n

    x_train_chunk = x_train[start:end] 
    y_train_chunk = y_train[start:end]
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train_chunk, y_train_chunk, epochs=1, batch_size=32)
    train_loss, train_acc = model.evaluate(x_train_chunk, y_train_chunk, verbose=2) 
    train_acc = comm.allreduce(train_acc, op=MPI.SUM)
    model.compile??
    
    
    return train_acc / size

model.compile

Object `model.compile` not found.


In [5]:
epochs = 5

for epoch in range(epochs): 
    
    train_acc = train(model, x_train, y_train, rank, size) 
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2) 
    test_acc = comm.allreduce(test_acc, op=MPI.SUM)

    if rank == 0:
        print(f"Epoch {epoch + 1}: Train accuracy = {train_acc:.4f}, Test accuracy = {test_acc / size:.4f}")

1875/1875 [==============================] - 42s 20ms/step - loss: 0.2107 - accuracy: 0.9412
1875/1875 - 14s - loss: 0.0941 - accuracy: 0.9727 - 14s/epoch - 7ms/step
313/313 - 3s - loss: 0.0975 - accuracy: 0.9710 - 3s/epoch - 8ms/step
Epoch 1: Train accuracy = 0.9727, Test accuracy = 0.9710
1875/1875 [==============================] - 41s 21ms/step - loss: 0.0800 - accuracy: 0.9764
1875/1875 - 14s - loss: 0.0574 - accuracy: 0.9834 - 14s/epoch - 7ms/step
313/313 - 2s - loss: 0.0637 - accuracy: 0.9789 - 2s/epoch - 8ms/step
Epoch 2: Train accuracy = 0.9834, Test accuracy = 0.9789
1875/1875 [==============================] - 37s 19ms/step - loss: 0.0600 - accuracy: 0.9819
1875/1875 - 13s - loss: 0.0477 - accuracy: 0.9866 - 13s/epoch - 7ms/step
313/313 - 3s - loss: 0.0614 - accuracy: 0.9807 - 3s/epoch - 8ms/step
Epoch 3: Train accuracy = 0.9866, Test accuracy = 0.9807
1875/1875 [==============================] - 39s 20ms/step - loss: 0.0493 - accuracy: 0.9852
1875/1875 - 13s - loss: 0.0432 

In [ ]:
model.c